In [ ]:
import gym
from gym import spaces
import numpy as np
import torch
from sklearn.metrics import r2_score
from scipy import stats

import xgboost as xgb
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import gym
from gym import spaces
import torch
import numpy as np
from scipy import stats


from stable_baselines3.common.vec_env import DummyVecEnv


from sb3_contrib import ARS # 機率輸出不會改變
from stable_baselines3 import A2C # 機率輸出會改變
from stable_baselines3 import DDPG # 輸出機率和動作長依樣?
from stable_baselines3 import HER # 這要搭配其他使用
from stable_baselines3 import PPO # 機率輸出會改變
from stable_baselines3 import SAC # 機率輸出會改變
from stable_baselines3 import TD3 # 輸出機率和動作長依樣?
from sb3_contrib import TRPO
from sb3_contrib import TQC
from sb3_contrib import RecurrentPPO # MlpLstmPolicy

In [ ]:
# Generate synthetic data
n_samples = 5000
n_features = 53
n_informative = 53
n_targets = 27
noise = 0.1
random_seed = np.random.randint(0, 1000)[0]

X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_targets=n_targets, noise=noise, random_state=random_seed)

reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed) # tree_method="hist", device="cuda"
reg.fit(X_train, y_train)


In [ ]:
import gym
from gym import spaces
import numpy as np
import torch
from sklearn.metrics import r2_score
from scipy import stats

class CustomEnv(gym.Env):
    def __init__(self, S, A, model, max_step=100, device='cpu'):
        self.device = device
        
        # Normalize S and A
        self.S = torch.tensor(S, dtype=torch.float32).to(self.device)#self._normalize(torch.tensor(S, dtype=torch.float32)).to(self.device)
        self.A = torch.tensor(A, dtype=torch.float32).to(self.device)#self._normalize(torch.tensor(A, dtype=torch.float32)).to(self.device)
        
        self.num_samples = self.S.shape[0]
        self.current_step = 0
        self.model = model
        self.max_step = max_step
        self.action_instance = self.A.shape[0]
        self.action_dim = self.A.shape[1]
        
        # 计算边界（假设数据大致遵循正态分布）
        self.obs_low = float(self.S.min())*1.5
        self.obs_high = float(self.S.max())*1.5
        self.action_low = float(self.A.min())*1.5
        self.action_high = float(self.A.max())*1.5

        # Limit the action space
        self.action_space = spaces.Box(low=self.action_low, high=self.action_high, shape=(self.action_instance, self.action_dim), dtype=np.float32)
        
        self.obs_dim_S = self.S.shape[1]
        self.observation_space = spaces.Box(low=self.obs_low, high=self.obs_high, shape=(self.num_samples, self.obs_dim_S), dtype=np.float32)
        
    # def _normalize(self, tensor):
    #     mean = tensor.mean(dim=0, keepdim=True)
    #     std = tensor.std(dim=0, keepdim=True)
    #     return (tensor - mean) / (std + 1e-8)  # Add small epsilon to avoid division by zero
    
    # def _denormalize(self, tensor, original):
    #     mean = original.mean(dim=0, keepdim=True)
    #     std = original.std(dim=0, keepdim=True)
    #     return tensor * (std + 1e-8) + mean

    def _get_observation(self):
        # self.current_step 放哪?
        num_instance = self.S.shape[0]
        random_index = np.random.randint(0,num_instance, size=int(self.num_samples))
        obs = self.S[random_index,:]#.shape 
        #print("DONE 2")    
        return obs, random_index
    def reset(self):
        self.current_step = 0
        obs , _ = self._get_observation()
        #print("DONE 3")  
        return obs
    
    def step(self, action):
        # 确保动作在边界内
        #action = np.clip(action, -1, 1)
        next_state, index = self._get_observation()
        action = torch.tensor(action, dtype=torch.float32).to(self.device)
        #print(action.shape, self.A.shape)
        # 在使用之前反标准化动作
        denorm_action = action[index,:]#self._denormalize(action, self.A)
        
        reward, score = self._calculate_reward(denorm_action)
        #print("STEP.....","reward: ",reward,"score: ",score)
        best_action = None
        best_score = float('-inf')
        if reward > 0 and score is not None:
            # 如果 score 是张量，取其平均值
            current_score = score.mean().item() if isinstance(score, torch.Tensor) else score
            if current_score > best_score:
                best_score = current_score
                best_action = denorm_action.cpu().numpy()  # 将最佳动作转换为 numpy 数组

        done = (self.current_step == self.max_step - 1)
        self.current_step += 1
        
        return next_state.cpu().numpy(), reward, done, {"best_action": best_action, "best_score": best_score}
    
    def _calculate_reward(self, action):
        try:
            next_state, index = self._get_observation()

            #print(action.shape)
            #print(self.A.shape)
            #print(len(index))
            y_pred_observe = torch.tensor(self.model.predict(action[index,:].cpu().numpy()), dtype=torch.float32).to(self.device)
            y_pred = torch.tensor(self.model.predict(self.A[index,:].cpu().numpy()), dtype=torch.float32).to(self.device)

            # 反标准化预测和真实值以进行正确的 R2 计算
            # denorm_S = self._denormalize(self.S, self.S)
            # denorm_y_pred_observe = self._denormalize(y_pred_observe, self.S)
            # denorm_y_pred = self._denormalize(y_pred, self.S)

            r2_observe = r2_score(next_state.cpu().numpy(), y_pred_observe.cpu().numpy(), multioutput='raw_values')
            r2_ground = r2_score(next_state.cpu().numpy(), y_pred.cpu().numpy(), multioutput='raw_values')

            # 检查 NaN 值
            if np.isnan(r2_observe).any() or np.isnan(r2_ground).any():
                return 0.0, None

            score = None
            #print("r2_observe: ",r2_observe,"r2_ground: ",r2_ground)
            if stats.ttest_ind(r2_observe, r2_ground, alternative="greater").pvalue < 0.05:
                reward = float(np.mean(r2_observe) - np.mean(r2_ground))  # 连续奖励
                score = r2_observe
                
            elif stats.ttest_ind(r2_observe, r2_ground, alternative="less").pvalue < 0.05:
                reward = float(np.mean(r2_observe) - np.mean(r2_ground))  # 负奖励
            else:
                reward = 0.0
            
            return reward, torch.tensor(score).to(self.device) if score is not None else None
        
        except Exception as e:
            print(f"Error in _calculate_reward: {e}")
            return 0.0, None